## <center><font color='black'> DEEPTHI SUDHARSAN (ROLL NUMBER : CB.EN.U4AIE19022)</font></center>
## <center><font color='black'> AI IN NLP ASSIGNMENT 4 </font></center>

#### The following sentences can be used to find the transition and emission probabilities.
•	It/PRP looks/VB like/IN circle/N 
•	He/PRP like/IN chocolate/N
•	I/PRP like/VB chocolate/N

#### The following sentence can used to find the maximum probable POS tags 

‘It looks like chocolate’


In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
import itertools

### 1. Write a python code to find the probability of word sequence for a part-of-speech sequence. 

In [2]:
# Emission Probability Function Definition
def emission_probability(vocab,tags,inp_text):
    temp_inp = [item for sublist in inp_text for item in sublist]
    # initialize emission probability matrix with zeros
    em_prob = np.zeros((len(vocab),len(tags)))
    # whenever we come across a particular word and its tag, we add 1 to that particular location
    # for that word and tag in the emission probability matrix
    for i in range(len(vocab)):
        for j in range(len(tags)):
            for k in temp_inp:
                if(k[0] == vocab[i] and k[1] == tags[j]):
                    em_prob[i][j] = em_prob[i][j] + 1
    # Converting the emission probability ndarray to a dataframe
    em_prob_df = pd.DataFrame(em_prob, columns = tags)
    # inserting a column at the beginning to represent the words
    em_prob_df.insert(0, "Words", vocab)
    # divide the values in each tag column by that column sum to get the emission probabilities
    for col in tags:
        em_prob_df[col] = em_prob_df[col]/em_prob_df[col].sum()
    # resetting the index to the words/vocab instead of index numbers
    em_prob_df.set_index('Words',inplace = True)
    return em_prob_df

def transition_probability(vocab,tags,inp_text):
    tags_modified = tags.copy()
    # inserting start and end of sentence tags
    tags_modified.insert(0,"<s>")
    tags_modified.insert(len(tags_modified) + 1,"</s>")
    # supressing 0/0 warning
    np.seterr(divide='ignore', invalid='ignore')
    # initializing the transition probability matrix with zeros
    trans_prob = np.zeros((len(tags_modified),len(tags_modified)))
    # for each sentence, if a particular tag is being followed by some other tag, we add 1 to the 
    # corresponding element in the transition matrix
    # if we are i nthe first word of the sentence then we need to also add 1 to the transition
    # from start of sentence tag to the first word
    # similarly we do the same when we reach the end of the sentence and update the transition matrix
    # position for the last word to </s> end of sentence tag transition
    for sent in inp_text:
        for s in range(len(sent) - 1):
            if(s == 0):
                trans_prob[tags_modified.index("<s>")][tags_modified.index(sent[s][1])] += 1
            trans_prob[tags_modified.index(sent[s][1])][tags_modified.index(sent[s+1][1])] += 1
        trans_prob[tags_modified.index(sent[len(sent)-1][1])][tags_modified.index("</s>")] += 1
    # now that we have the count, we divide each tag row with the row sum to get the transition probabilities
    trans_prob = trans_prob/trans_prob.sum(axis=1, keepdims=True)
    # if there were any 0/0 division areas, we just change the value from nan to 0
    trans_prob = np.nan_to_num(trans_prob)
    # converting ndarray to dataframe
    trans_prob_df = pd.DataFrame(trans_prob, columns = tags_modified)
    # inserting tags column with the tags list and setting it as index
    trans_prob_df.insert(0, "Tags", tags_modified)
    trans_prob_df.set_index("Tags",inplace = True)
    return trans_prob_df

In [3]:
def prob_word_given_pos(word_seq,pos_seq,e,t):
    # splitting the words and tags from the word and pos sequences
    words = word_seq.split()
    pos = pos_seq.split()
    # creating an array to store the state probabilities for each pos state given
    probs = np.zeros(len(words))
#     total_prob = t[pos[0]]["<s>"] #+ t["</s>"][pos[-1]]
    for i in range(len(words)):
        # storing the emission probability first
        probs[i] = e[pos[i]][words[i]]
        # multiplying with the transition probaility
        # if we are in the last word of the sentence we need to multiply the transition probabilty of that word to 
        # the end of sentence tag
        if(i != len(words)-1):
            probs[i] = probs[i] * t[pos[i+1]][pos[i]]
        else:
            probs[i] = probs[i] * t["</s>"][pos[i]]
    # returning the product of the all the state probailities and multiplying it with the start probability
    return t[pos[0]]["<s>"] * np.prod(probs)

In [4]:
# sample sentences given to find the transition and emission probabilities
sample_text = ["It/PRP looks/VB like/IN circle/N","He/PRP like/IN chocolate/N","I/PRP like/VB chocolate/N"]
inp = []
# splitting each sentence in the list to a list of words based on space delimiter
for st in sample_text:
    inp.append(st.split())
# converting each list of words to tuple pairs consisting of the word and its tag by splitting based on /
inp_text = []
for ws in inp:
    temp = []
    for w in ws:
        temp.append(tuple(w.split('/')))
    inp_text.append(temp)    
inp_text     

[[('It', 'PRP'), ('looks', 'VB'), ('like', 'IN'), ('circle', 'N')],
 [('He', 'PRP'), ('like', 'IN'), ('chocolate', 'N')],
 [('I', 'PRP'), ('like', 'VB'), ('chocolate', 'N')]]

In [5]:
# getting a list of unique tags and words
tags = set()
vocab = set()
for i in inp_text:
    for tp in i:
        tags.add(tp[1])
        vocab.add(tp[0])
tags = list(tags)
vocab = list(vocab)
print(tags)
print(vocab)

['N', 'IN', 'PRP', 'VB']
['It', 'chocolate', 'I', 'like', 'looks', 'He', 'circle']


In [6]:
# calling and storing the emission probability dataframe
e = emission_probability(vocab,tags,inp_text)
e

,N,IN,PRP,VB
Words,,,,
It,0.000000,0.0,0.333333,0.0
chocolate,0.666667,0.0,0.000000,0.0
I,0.000000,0.0,0.333333,0.0
like,0.000000,1.0,0.000000,0.5
looks,0.000000,0.0,0.000000,0.5
He,0.000000,0.0,0.333333,0.0
circle,0.333333,0.0,0.000000,0.0


In [7]:
# calling and storing the transition probability dataframe
t = transition_probability(vocab,tags,inp_text)
t

,<s>,N,IN,PRP,VB,</s>
Tags,,,,,,
<s>,0.0,0.0,0.000000,1.0,0.000000,0.0
N,0.0,0.0,0.000000,0.0,0.000000,1.0
IN,0.0,1.0,0.000000,0.0,0.000000,0.0
PRP,0.0,0.0,0.333333,0.0,0.666667,0.0
VB,0.0,0.5,0.500000,0.0,0.000000,0.0
</s>,0.0,0.0,0.000000,0.0,0.000000,0.0


In [8]:
word_seq = "It looks like chocolate"
pos_seq = "PRP VB IN N"
# calling the function to calculate the proability of a word sequence given the pos sequence
prob_word_given_pos(word_seq,pos_seq,e,t)

0.037037037037037035

In [9]:
word_seq = "I like"
pos_seq = "PRP VB"
# calling the function to calculate the probability of a word sequence given the pos sequence
prob_word_given_pos(word_seq,pos_seq,e,t)

0.0

### 2. Display the possible combinations (brute-force) and its probability value.

In [10]:
word_seq = "It looks like chocolate"
# all combinations of the words in the given input word sequence with repetition (sentence size = length of input
# word sequence)
diff_tags_combos = list(combinations_with_replacement(tags, len(word_seq.split())))
# all combinations of the words in the given input word sequence without repetition (sentence size = length of input
# word sequence) and appending these combinations to the above combinations list
diff_tags_combos.extend(list(itertools.permutations(tags)))
pos_seq = ""
prob_combo = {}
combo_str = ""
# for each of the combinations we call the function to calculate the probability of the word sequence 

for combo in diff_tags_combos:
    pos_seq = ' '.join(combo)
    combo_str = "P( " + word_seq + " | " + pos_seq + " )"
    prob_combo[combo_str] = prob_word_given_pos(word_seq,pos_seq,e,t)
# printing all possible combinations and their probabilities neatly
for key, value in prob_combo.items():
    print(key, ' : ', value)

P( It looks like chocolate | N N N N )  :  0.0
P( It looks like chocolate | N N N IN )  :  0.0
P( It looks like chocolate | N N N PRP )  :  0.0
P( It looks like chocolate | N N N VB )  :  0.0
P( It looks like chocolate | N N IN IN )  :  0.0
P( It looks like chocolate | N N IN PRP )  :  0.0
P( It looks like chocolate | N N IN VB )  :  0.0
P( It looks like chocolate | N N PRP PRP )  :  0.0
P( It looks like chocolate | N N PRP VB )  :  0.0
P( It looks like chocolate | N N VB VB )  :  0.0
P( It looks like chocolate | N IN IN IN )  :  0.0
P( It looks like chocolate | N IN IN PRP )  :  0.0
P( It looks like chocolate | N IN IN VB )  :  0.0
P( It looks like chocolate | N IN PRP PRP )  :  0.0
P( It looks like chocolate | N IN PRP VB )  :  0.0
P( It looks like chocolate | N IN VB VB )  :  0.0
P( It looks like chocolate | N PRP PRP PRP )  :  0.0
P( It looks like chocolate | N PRP PRP VB )  :  0.0
P( It looks like chocolate | N PRP VB VB )  :  0.0
P( It looks like chocolate | N VB VB VB )  :  0.0


### 3. Implement Viterbi to find the maximum probable POS tag sequence

In [11]:
# training tagged data 
train_data = [item for sublist in inp_text for item in sublist]
train_data

[('It', 'PRP'),
 ('looks', 'VB'),
 ('like', 'IN'),
 ('circle', 'N'),
 ('He', 'PRP'),
 ('like', 'IN'),
 ('chocolate', 'N'),
 ('I', 'PRP'),
 ('like', 'VB'),
 ('chocolate', 'N')]

In [12]:
def Viterbi(words, train_bag = train_data):
    # words is the untagged word sequence we want to find the pos sequence of
    state = []
    # list to store the states that achieved max probability
    T = list(set([pair[1] for pair in train_bag]))
    # list of all the tags from the training data
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        # for each tag in the tag list
        for tag in T:
            # if key is 0, we are in the first word of the word sequence so we need to get transition from start
            # tag to the first word of the sentence
            # else compute transition probabilities based on the prev state to current state
            if key == 0:
                transition_p = t[tag]["<s>"]
            else:
                transition_p = t[tag][state[-1]]
            # emission probability
            emission_p = e[tag][words[key]]
            # state probability
            state_probability = emission_p * transition_p
            # appending the state probabilities
            p.append(state_probability) 
        # getting the maximum state probability and finding out which state the probability belongs to
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        # appending the state with max probability
        state.append(state_max)
    # return the respective word and its state/tag
    return list(zip(words, state))

In [13]:
word_seq = "It looks like chocolate"
print("Maximum probable POS tag sequence for the word sequence",word_seq,"is : ")
Viterbi(word_seq.split())

Maximum probable POS tag sequence for the word sequence It looks like chocolate is : 


[('It', 'PRP'), ('looks', 'VB'), ('like', 'IN'), ('chocolate', 'N')]